
<h1 align=center><font size = 6> Segmenting and Clustering Neighborhoods in Toronto. </font></h1>

# Name : AKSHAY DILIP GODSE.

### Import required libraries.

In [29]:
import numpy as np    #Library to handle data in vectorized manner.
import pandas as pd   #library for data analysis.
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from geopy.geocoders import Nominatim    #convert an address into latitude and longitude address.
import requests    #library to handle requests.

from pandas.io.json import json_normalize  #transform json file into pandas dataframe.
import json

#Plotting libray matplotlib and associated ploting modules.
import matplotlib.cm as cm
import matplotlib.colors as colors

import folium # map rendering library

# import k-means.
from sklearn.cluster import KMeans

# import Beautiful Soup for web scrapping.
from bs4 import BeautifulSoup
import lxml

%matplotlib inline

print('All the required libraries are imported.....!')

All the required libraries are imported.....!


### To acquire data, First we need to scrape the wikipedia page.

In [30]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'lxml')

### Locate table and find the Postal Code by using Borough and Neighbourhood tags.

In [31]:
Postalcode_table = soup.find('table')
fields = Postalcode_table.find_all('td')    #find the table data.

# Define the table columns.
postcode = []
borough = []
neighborhood = []

# Append the values into the tables.
for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighborhood.append(fields[i+2].text.strip())

#transform data into pandas data frame.    
df_postalcode = pd.DataFrame(data=[postcode, borough, neighborhood]).transpose()  
df_postalcode.columns = ['Postcode', 'Borough', 'Neighborhood']



In [32]:
#lets have a look at our dataframe.
df_postalcode.head()

Postcode           Borough                Neighborhood
0      M1A      Not assigned                            
1      M2A      Not assigned                            
2      M3A        North York                   Parkwoods
3      M4A        North York            Victoria Village
4      M5A  Downtown Toronto  Regent Park / Harbourfront

### Now let's get remove the Not Assigned fields from the dataframe.

In [33]:
df_postalcode['Borough'].replace('Not assigned', np.nan, inplace=True)
df_postalcode.dropna(subset=['Borough'], inplace=True)

In [34]:
# Lets have look into the dataframe after dropping Not Assigned or incomplete rows.
df_postalcode.head()

Postcode           Borough                                  Neighborhood
2      M3A        North York                                     Parkwoods
3      M4A        North York                              Victoria Village
4      M5A  Downtown Toronto                    Regent Park / Harbourfront
5      M6A        North York             Lawrence Manor / Lawrence Heights
6      M7A  Downtown Toronto  Queen's Park / Ontario Provincial Government

### Clean Dataframe : Which Postal code has more than two neighborhoods, combine these in a single row by using neighborhoods seperated by comma's.

In [35]:
newdf_postalcode = df_postalcode.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
newdf_postalcode.columns  = ['Postcode', 'Borough', 'Neighborhood']

In [36]:
# Now lets have look how dataframe looks.
newdf_postalcode.head()


Postcode      Borough                              Neighborhood
0      M1B  Scarborough                           Malvern / Rouge
1      M1C  Scarborough  Rouge Hill / Port Union / Highland Creek
2      M1E  Scarborough       Guildwood / Morningside / West Hill
3      M1G  Scarborough                                    Woburn
4      M1H  Scarborough                                 Cedarbrae

### Assign the neighborhood same as Borough for which Borouh has Not Assigned Neighborhood.

In [37]:
newdf_postalcode.loc[newdf_postalcode['Neighborhood'] =='Not assigned' , 'Neighborhood'] = newdf_postalcode['Borough']

#lets have look on the dataframe.
newdf_postalcode

Postcode           Borough  \
0        M1B       Scarborough   
1        M1C       Scarborough   
2        M1E       Scarborough   
3        M1G       Scarborough   
4        M1H       Scarborough   
5        M1J       Scarborough   
6        M1K       Scarborough   
7        M1L       Scarborough   
8        M1M       Scarborough   
9        M1N       Scarborough   
10       M1P       Scarborough   
11       M1R       Scarborough   
12       M1S       Scarborough   
13       M1T       Scarborough   
14       M1V       Scarborough   
15       M1W       Scarborough   
16       M1X       Scarborough   
17       M2H        North York   
18       M2J        North York   
19       M2K        North York   
20       M2L        North York   
21       M2M        North York   
22       M2N        North York   
23       M2P        North York   
24       M2R        North York   
25       M3A        North York   
26       M3B        North York   
27       M3C        North York   
28       M3H        North York   
29       M3J        North York   
30       M3K        North York   
31       M3L        North York   
32       M3M        North York   
33       M3N        North York   
34       M4A        North York   
35       M4B         East York   
36       M4C         East York   
37       M4E      East Toronto   
38       M4G         East York   
39       M4H         East York   
40       M4J         East York   
41       M4K      East Toronto   
42       M4L      East Toronto   
43       M4M      East Toronto   
44       M4N   Central Toronto   
45       M4P   Central Toronto   
46       M4R   Central Toronto   
47       M4S   Central Toronto   
48       M4T   Central Toronto   
49       M4V   Central Toronto   
50       M4W  Downtown Toronto   
51       M4X  Downtown Toronto   
52       M4Y  Downtown Toronto   
53       M5A  Downtown Toronto   
54       M5B  Downtown Toronto   
55       M5C  Downtown Toronto   
56       M5E  Downtown Toronto   
57       M5G  Downtown Toronto   
58       M5H  Downtown Toronto   
59       M5J  Downtown Toronto   
60       M5K  Downtown Toronto   
61       M5L  Downtown Toronto   
62       M5M        North York   
63       M5N   Central Toronto   
64       M5P   Central Toronto   
65       M5R   Central Toronto   
66       M5S  Downtown Toronto   
67       M5T  Downtown Toronto   
68       M5V  Downtown Toronto   
69       M5W  Downtown Toronto   
70       M5X  Downtown Toronto   
71       M6A        North York   
72       M6B        North York   
73       M6C              York   
74       M6E              York   
75       M6G  Downtown Toronto   
76       M6H      West Toronto   
77       M6J      West Toronto   
78       M6K      West Toronto   
79       M6L        North York   
80       M6M              York   
81       M6N              York   
82       M6P      West Toronto   
83       M6R      West Toronto   
84       M6S      West Toronto   
85       M7A  Downtown Toronto   
86       M7R       Mississauga   
87       M7Y      East Toronto   
88       M8V         Etobicoke   
89       M8W         Etobicoke   
90       M8X         Etobicoke   
91       M8Y         Etobicoke   
92       M8Z         Etobicoke   
93       M9A         Etobicoke   
94       M9B         Etobicoke   
95       M9C         Etobicoke   
96       M9L        North York   
97       M9M        North York   
98       M9N              York   
99       M9P         Etobicoke   
100      M9R         Etobicoke   
101      M9V         Etobicoke   
102      M9W         Etobicoke   

                                          Neighborhood  
0                                      Malvern / Rouge  
1             Rouge Hill / Port Union / Highland Creek  
2                  Guildwood / Morningside / West Hill  
3                                               Woburn  
4                                            Cedarbrae  
5                                  Scarborough Village  
6        Kennedy Park / Ionview / East Birchmount Park  
7          

In [38]:
# reanme the column name PostCode as Postal Code.
newdf_postalcode.rename(columns={"Postcode": "Postal Code"}, inplace=True)
newdf_postalcode.head()

Postal Code      Borough                              Neighborhood
0         M1B  Scarborough                           Malvern / Rouge
1         M1C  Scarborough  Rouge Hill / Port Union / Highland Creek
2         M1E  Scarborough       Guildwood / Morningside / West Hill
3         M1G  Scarborough                                    Woburn
4         M1H  Scarborough                                 Cedarbrae

In [39]:
### Use .shape method to print the number of of the dataframe.
newdf_postalcode.shape

(103, 3)

### Now load the Geospatial co-ordinates data csv file using pandas dataframe.

In [41]:
data_coordinates = pd.read_csv('https://cocl.us/Geospatial_data')

#lets have look into co-ordinates dataframe.
data_coordinates.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

### Now merge the above two dataframe into a single dataframe.


In [42]:
df_toronto = newdf_postalcode.merge(data_coordinates, on="Postal Code", how="left")

#lets have a look on dataframe.
df_toronto.head()

Postal Code      Borough                              Neighborhood  \
0         M1B  Scarborough                           Malvern / Rouge   
1         M1C  Scarborough  Rouge Hill / Port Union / Highland Creek   
2         M1E  Scarborough       Guildwood / Morningside / West Hill   
3         M1G  Scarborough                                    Woburn   
4         M1H  Scarborough                                 Cedarbrae   

    Latitude  Longitude  
0  43.806686 -79.194353  
1  43.784535 -79.160497  
2  43.763573 -79.188711  
3  43.770992 -79.216917  
4  43.773136 -79.239476

### Now lets get the geographical co-ordinates of the Toronto City.

In [45]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


### Now Create Map of Toronto City and Add markers.

In [46]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.4,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Let's Define the Foursquare API Credentials and Version.

In [47]:
CLIENT_ID = 'ZPAHF5PIW1FTZM0PWZTUDAZ205BSOHY2TYCPY4JUJ0KDBPS4' # your Foursquare ID
CLIENT_SECRET = 'EF1Q0BA3KKUCVJNI2KPTGQIF3IQ5UCPZPB5OODLX4E1CTLCP' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZPAHF5PIW1FTZM0PWZTUDAZ205BSOHY2TYCPY4JUJ0KDBPS4
CLIENT_SECRET:EF1Q0BA3KKUCVJNI2KPTGQIF3IQ5UCPZPB5OODLX4E1CTLCP


### Explore Neighborhoods in Toronto City.

In [48]:
# explore the first neighborhood.
df_toronto.loc[0, 'Neighborhood']

'Malvern / Rouge'

In [49]:
#lets see the values of longitude and latitude values of neighborhoods.
neigh_lat = df_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neigh_long = df_toronto.loc[0, 'Longitude'] # neighborhood longitude value

neigh_name = df_toronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neigh_name, 
                                                               neigh_lat, 
                                                               neigh_long))

Latitude and longitude values of Malvern / Rouge are 43.806686299999996, -79.19435340000001.


Lets Create the GET request and name your URL.

In [50]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neigh_lat, 
    neigh_long, 
    radius, 
    LIMIT)
url # display URL




'https://api.foursquare.com/v2/venues/explore?&client_id=ZPAHF5PIW1FTZM0PWZTUDAZ205BSOHY2TYCPY4JUJ0KDBPS4&client_secret=EF1Q0BA3KKUCVJNI2KPTGQIF3IQ5UCPZPB5OODLX4E1CTLCP&v=20180604&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

Send GET request and examin the results.

In [51]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e9f0be56001fe001cac61f2'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': 'Wendy’s',
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [52]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json structure into pandas dataframe.

In [53]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

name            categories        lat        lng
0  Wendy’s  Fast Food Restaurant  43.807448 -79.199056

Print how many venues returned by foursquare.

In [54]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


## Now lets explore the toronto city.

Create function to repeat the process to all toronto neighborhoods.

In [55]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now Run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [56]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )



Malvern / Rouge
Rouge Hill / Port Union / Highland Creek
Guildwood / Morningside / West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park / Ionview / East Birchmount Park
Golden Mile / Clairlea / Oakridge
Cliffside / Cliffcrest / Scarborough Village West
Birch Cliff / Cliffside West
Dorset Park / Wexford Heights / Scarborough Town Centre
Wexford / Maryvale
Agincourt
Clarks Corners / Tam O'Shanter / Sullivan
Milliken / Agincourt North / Steeles East / L'Amoreaux East
Steeles West / L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview / Henry Farm / Oriole
Bayview Village
York Mills / Silver Hills
Willowdale / Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Bathurst Manor / Wilson Heights / Downsview North
Northwood Park / York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill / Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West / Riverdale
India Bazaar / The Beaches 

let's check the size of the reulting dataframe.

In [57]:
print(toronto_venues.shape)
toronto_venues.head()

(2130, 7)


Neighborhood  Neighborhood Latitude  \
0                           Malvern / Rouge              43.806686   
1  Rouge Hill / Port Union / Highland Creek              43.784535   
2       Guildwood / Morningside / West Hill              43.763573   
3       Guildwood / Morningside / West Hill              43.763573   
4       Guildwood / Morningside / West Hill              43.763573   

   Neighborhood Longitude                  Venue  Venue Latitude  \
0              -79.194353                Wendy’s       43.807448   
1              -79.160497  Royal Canadian Legion       43.782533   
2              -79.188711      G & G Electronics       43.765309   
3              -79.188711       Big Bite Burrito       43.766299   
4              -79.188711  Enterprise Rent-A-Car       43.764076   

   Venue Longitude        Venue Category  
0       -79.199056  Fast Food Restaurant  
1       -79.163085                   Bar  
2       -79.191537     Electronics Store  
3       -79.190720    Mexican Restaurant  
4       -79.193406   Rental Car Location

lets check how many venues are returned for each neighborhood.

In [58]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Agincourt                                                               4   
Alderwood / Long Branch                                                 9   
Bathurst Manor / Wilson Heights / Downsview North                      20   
Bayview Village                                                         4   
Bedford Park / Lawrence Manor East                                     23   
Berczy Park                                                            57   
Birch Cliff / Cliffside West                                            4   
Brockton / Parkdale Village / Exhibition Place                         23   
Business reply mail Processing CentrE                                  17   
CN Tower / King and Spadina / Railway Lands / H...                     16   
Caledonia-Fairbanks                                                     4   
Canada Post Gateway Processing Centre                                  13   
Cedarbrae                                                               8   
Central Bay Street                                                     65   
Christie                                                               18   
Church and Wellesley                                                   72   
Clarks Corners / Tam O'Shanter / Sullivan                              16   
Cliffside / Cliffcrest / Scarborough Village West                       2   
Commerce Court / Victoria Hotel                                       100   
Davisville                                                             34   
Davisville North                                                        7   
Del Ray / Mount Dennis / Keelsdale and Silverthorn                      4   
Don Mills                                                              27   
Dorset Park / Wexford Heights / Scarborough Tow...                      5   
Downsview                                                              16   
Dufferin / Dovercourt Village                                          16   
East Toronto                                                            3   
Eringate / Bloordale Gardens / Old Burnhamthorp...                      9   
Fairview / Henry Farm / Oriole                                         67   
First Canadian Place / Underground city                               100   
Forest Hill North & West                                                4   
Garden District, Ryerson                                              100   
Glencairn                                                               4   
Golden Mile / Clairlea / Oakridge                                       8   
Guildwood / Morningside / West Hill                                     7   
Harbourfront East / Union Station / Toronto Isl...                    100   
High Park / The Junction South                                         23   
Hillcrest Village                                                       5   
Humber Summit                                                           3   
Humberlea / Emery                                                       2   
Humewood-Cedarvale                                                      5   
India Bazaar / The Beaches West                                        21   
Kennedy Park / Ionview / East Birchmount Park                           6   
Kensington Market / Chinatown / Grange Park                            56   
Kingsview Village / St. Phillips / Martin Grove...                      4   
Lawrence Manor / Lawrence Heights                                      14   
Lawrence Park                                                           3   
Leaside                                                                34   
Little Portugal / Trinity                                              42   
Malvern / Rouge                                                         1   
Milliken / Agincourt North / Steeles East / L'A.

Let's find out how many unique categories can be curated from all the returned venues.

In [59]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 271 uniques categories.


### Now Ananlyze each nighborhood.

In [60]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Yoga Studio  Accessories Store  Airport  Airport Food Court  Airport Gate  \
0            0                  0        0                   0             0   
1            0                  0        0                   0             0   
2            0                  0        0                   0             0   
3            0                  0        0                   0             0   
4            0                  0        0                   0             0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Antique Shop  Aquarium  Art Gallery  Art Museum  Arts & Crafts Store  \
0             0         0            0           0                    0   
1             0         0            0           0                    0   
2             0         0            0           0                    0   
3             0         0            0           0                    0   
4             0         0            0           0                    0   

   Asian Restaurant  Athletics & Sports  Auto Workshop  BBQ Joint  Baby Store  \
0                 0                   0              0          0           0   
1                 0                   0              0          0           0   
2                 0                   0              0          0           0   
3                 0                   0              0          0           0   
4                 0                   0              0          0           0   

   Bagel Shop  Bakery  Bank  Bar  Baseball Field  Baseball Stadium  \
0           0       0     0    0               0                 0   
1           0       0     0    1               0                 0   
2           0       0     0    0               0                 0   
3           0       0     0    0               0                 0   
4           0       0     0    0               0                 0   

   Basketball Court  Basketball Stadium  Beach  Bed & Breakfast  Beer Bar  \
0                 0                   0      0                0         0   
1                 0                   0      0                0         0   
2                 0                   0      0                0         0   
3                 0                   0      0                0         0   
4                 0                   0      0                0         0   

   Beer Store  Belgian Restaurant  Bike Shop  Bistro  Boat or Ferry  \
0           0                   0          0       0              0   
1           0                   0          0       0              0   
2           0                   0          0       0              0   
3           0                   0          0       0              0   
4           0                   0          0       0              0   

   Bookstore  Boutique  Brazilian Restaurant  Breakfast Spot  Brewery  \
0          0         0                     0               0        0   
1          0         0                     0               0        0   
2          0         0                     0               0        0   
3          0         0                     0               0        0   
4          0         0                     0               0        0   

   Bridal Shop  Bubble Tea Shop  Building  Burger Joint  Burrito Place  \
0            0                0         0             0              0   
1            0                0         0             0              0   
2            0                0         0             0              0   
3            0                0         0             0              0   
4

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [61]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Neighborhood  Yoga Studio  \
0                                           Agincourt     0.000000   
1                             Alderwood / Long Branch     0.000000   
2   Bathurst Manor / Wilson Heights / Downsview North     0.000000   
3                                     Bayview Village     0.000000   
4                  Bedford Park / Lawrence Manor East     0.000000   
5                                         Berczy Park     0.000000   
6                        Birch Cliff / Cliffside West     0.000000   
7      Brockton / Parkdale Village / Exhibition Place     0.000000   
8               Business reply mail Processing CentrE     0.000000   
9   CN Tower / King and Spadina / Railway Lands / ...     0.000000   
10                                Caledonia-Fairbanks     0.000000   
11              Canada Post Gateway Processing Centre     0.000000   
12                                          Cedarbrae     0.000000   
13                                 Central Bay Street     0.015385   
14                                           Christie     0.000000   
15                               Church and Wellesley     0.027778   
16          Clarks Corners / Tam O'Shanter / Sullivan     0.000000   
17  Cliffside / Cliffcrest / Scarborough Village West     0.000000   
18                    Commerce Court / Victoria Hotel     0.000000   
19                                         Davisville     0.000000   
20                                   Davisville North     0.000000   
21  Del Ray / Mount Dennis / Keelsdale and Silvert...     0.000000   
22                                          Don Mills     0.000000   
23  Dorset Park / Wexford Heights / Scarborough To...     0.000000   
24                                          Downsview     0.000000   
25                      Dufferin / Dovercourt Village     0.000000   
26                                       East Toronto     0.000000   
27  Eringate / Bloordale Gardens / Old Burnhamthor...     0.000000   
28                     Fairview / Henry Farm / Oriole     0.000000   
29            First Canadian Place / Underground city     0.000000   
30                           Forest Hill North & West     0.000000   
31                           Garden District, Ryerson     0.000000   
32                                          Glencairn     0.000000   
33                  Golden Mile / Clairlea / Oakridge     0.000000   
34                Guildwood / Morningside / West Hill     0.000000   
35  Harbourfront East / Union Station / Toronto Is...     0.000000   
36                     High Park / The Junction South     0.000000   
37                                  Hillcrest Village     0.000000   
38                                      Humber Summit     0.000000   
39                                  Humberlea / Emery     0.000000   
40                                 Humewood-Cedarvale     0.000000   
41                    India Bazaar / The Beaches West     0.000000   
42      Kennedy Park / Ionview / East Birchmount Park     0.000000   
43        Kensington Market / Chinatown / Grange Park     0.000000   
44  Kingsview Village / St. Phillips / Martin Grov...     0.000000   
45                  Lawrence Manor / Lawrence Heights     0.000000   
46                                      Lawrence Park     0.000000   
47                                            Leaside     0.000000   
48                          Little Portugal / Trinity     0.023810   
49                                    Malvern / Rouge     0.000000   
50  Milliken / Agincourt North / Steeles East / L'...     0.000000   
51  Mimico NW / The Queensway West / South of Bloo...     0.000000   
52                       Moore Park / Summerhill East     0.000000   
53     New Toronto / Mimico South / Humber Bay Shores     0.000000   
54         North Park / Maple Leaf Park / Upwood Park     0.000000   
55                                 North Toronto West     0.050000   
56                                          Northwe

lets examine the new dataframe size.

In [62]:
toronto_onehot.shape

(2130, 271)

Lets print each neighborhood along with top five most common venues.

In [63]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1             Breakfast Spot  0.25
2               Skating Rink  0.25
3                     Lounge  0.25
4                Yoga Studio  0.00


----Alderwood / Long Branch----
                venue  freq
0         Pizza Place  0.22
1  Athletics & Sports  0.11
2         Coffee Shop  0.11
3                 Pub  0.11
4      Sandwich Place  0.11


----Bathurst Manor / Wilson Heights / Downsview North----
            venue  freq
0     Coffee Shop  0.10
1            Bank  0.10
2     Pizza Place  0.05
3   Shopping Mall  0.05
4  Sandwich Place  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1                 Bank  0.25
2   Chinese Restaurant  0.25
3  Japanese Restaurant  0.25
4  Moroccan Restaurant  0.00


----Bedford Park / Lawrence Manor East----
                     venue  freq
0              Coffee Shop  0.09
1               Restaurant  0.09
2           Sandwich P

                    venue  freq
0     Sporting Goods Shop  0.09
1             Coffee Shop  0.09
2            Burger Joint  0.06
3  Furniture / Home Store  0.06
4                    Bank  0.06


----Little Portugal / Trinity----
                           venue  freq
0                            Bar  0.10
1                     Restaurant  0.07
2                    Men's Store  0.05
3          Vietnamese Restaurant  0.05
4  Vegetarian / Vegan Restaurant  0.05


----Malvern / Rouge----
                             venue  freq
0             Fast Food Restaurant   1.0
1                      Men's Store   0.0
2  Molecular Gastronomy Restaurant   0.0
3       Modern European Restaurant   0.0
4                Mobile Phone Shop   0.0


----Milliken / Agincourt North / Steeles East / L'Amoreaux East----
                        venue  freq
0                        Park   0.5
1                  Playground   0.5
2                 Yoga Studio   0.0
3    Mediterranean Restaurant   0.0
4  Modern Europe

Lets put into the pandas dataframe.

First, let's write a function to sort the venues in descending order.

In [64]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [65]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood  \
0                                          Agincourt   
1                            Alderwood / Long Branch   
2  Bathurst Manor / Wilson Heights / Downsview North   
3                                    Bayview Village   
4                 Bedford Park / Lawrence Manor East   

       1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
0  Latin American Restaurant          Skating Rink                Lounge   
1                Pizza Place                   Gym           Coffee Shop   
2                Coffee Shop                  Bank      Sushi Restaurant   
3                       Bank    Chinese Restaurant   Japanese Restaurant   
4             Sandwich Place    Italian Restaurant            Restaurant   

  4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
0        Breakfast Spot            Donut Shop                 Diner   
1        Sandwich Place    Athletics & Sports                   Pub   
2           Pizza Place           Bridal Shop        Sandwich Place   
3                  Café         Women's Store      Doner Restaurant   
4           Coffee Shop             Juice Bar           Pizza Place   

  7th Most Common Venue    8th Most Common Venue 9th Most Common Venue  \
0        Discount Store      Distribution Center               Dog Run   
1          Skating Rink                 Pharmacy      Department Store   
2                 Diner               Restaurant        Ice Cream Shop   
3                 Diner           Discount Store   Distribution Center   
4              Boutique  Comfort Food Restaurant          Liquor Store   

        10th Most Common Venue  
0             Doner Restaurant  
1  Eastern European Restaurant  
2                Deli / Bodega  
3                      Dog Run  
4            Indian Restaurant

### Now lets Cluster the Neighborhoods.

Run k-means to cluster the neighborhood into 5 clusters.

In [66]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [70]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged  # check the last columns!

Postal Code           Borough  \
0           M1B       Scarborough   
1           M1C       Scarborough   
2           M1E       Scarborough   
3           M1G       Scarborough   
4           M1H       Scarborough   
5           M1J       Scarborough   
6           M1K       Scarborough   
7           M1L       Scarborough   
8           M1M       Scarborough   
9           M1N       Scarborough   
10          M1P       Scarborough   
11          M1R       Scarborough   
12          M1S       Scarborough   
13          M1T       Scarborough   
14          M1V       Scarborough   
15          M1W       Scarborough   
16          M1X       Scarborough   
17          M2H        North York   
18          M2J        North York   
19          M2K        North York   
20          M2L        North York   
21          M2M        North York   
22          M2N        North York   
23          M2P        North York   
24          M2R        North York   
25          M3A        North York   
26          M3B        North York   
27          M3C        North York   
28          M3H        North York   
29          M3J        North York   
30          M3K        North York   
31          M3L        North York   
32          M3M        North York   
33          M3N        North York   
34          M4A        North York   
35          M4B         East York   
36          M4C         East York   
37          M4E      East Toronto   
38          M4G         East York   
39          M4H         East York   
40          M4J         East York   
41          M4K      East Toronto   
42          M4L      East Toronto   
43          M4M      East Toronto   
44          M4N   Central Toronto   
45          M4P   Central Toronto   
46          M4R   Central Toronto   
47          M4S   Central Toronto   
48          M4T   Central Toronto   
49          M4V   Central Toronto   
50          M4W  Downtown Toronto   
51          M4X  Downtown Toronto   
52          M4Y  Downtown Toronto   
53          M5A  Downtown Toronto   
54          M5B  Downtown Toronto   
55          M5C  Downtown Toronto   
56          M5E  Downtown Toronto   
57          M5G  Downtown Toronto   
58          M5H  Downtown Toronto   
59          M5J  Downtown Toronto   
60          M5K  Downtown Toronto   
61          M5L  Downtown Toronto   
62          M5M        North York   
63          M5N   Central Toronto   
64          M5P   Central Toronto   
65          M5R   Central Toronto   
66          M5S  Downtown Toronto   
67          M5T  Downtown Toronto   
68          M5V  Downtown Toronto   
69          M5W  Downtown Toronto   
70          M5X  Downtown Toronto   
71          M6A        North York   
72          M6B        North York   
73          M6C              York   
74          M6E              York   
75          M6G  Downtown Toronto   
76          M6H      West Toronto   
77          M6J      West Toronto   
78          M6K      West Toronto   
79          M6L        North York   
80          M6M              York   
81          M6N              York   
82          M6P      West Toronto   
83          M6R      West Toronto   
84          M6S      West Toronto   
85          M7A  Downtown Toronto   
86          M7R       Mississauga   
87          M7Y      East Toronto   
88          M8V         Etobicoke   
89          M8W         Etobicoke   
90          M8X         Etobicoke   
91          M8Y         Etobicoke   
92          M8Z         Etobicoke   
93          M9A         Etobicoke   
94          M9B         Etobicoke   
95          M9C         Etobicoke   
96          M9L        North York   
97          M9M        North York   
98          M9N              York   
99          M9P         Etobicoke   
100         M9R         Etobicoke   
101         M9V         Etobicoke   
102         M9W         Etobicoke   

                                          Neighborhood   Latitude  Longitude  \
0                                      Malvern / Rouge  43.806686 -79.19435

Clean Dataframe drop Missing values and change the cluster labels type as integer.

In [71]:
toronto_merged.dropna(subset = ['Cluster Labels'], how = "all", inplace = True) #drop rows that contain NAN Values.

toronto_merged['Cluster Labels'] =toronto_merged['Cluster Labels'].astype(int)  #Change cluster Labels type as Int.

toronto_merged

Postal Code           Borough  \
0           M1B       Scarborough   
1           M1C       Scarborough   
2           M1E       Scarborough   
3           M1G       Scarborough   
4           M1H       Scarborough   
5           M1J       Scarborough   
6           M1K       Scarborough   
7           M1L       Scarborough   
8           M1M       Scarborough   
9           M1N       Scarborough   
10          M1P       Scarborough   
11          M1R       Scarborough   
12          M1S       Scarborough   
13          M1T       Scarborough   
14          M1V       Scarborough   
15          M1W       Scarborough   
17          M2H        North York   
18          M2J        North York   
19          M2K        North York   
20          M2L        North York   
21          M2M        North York   
22          M2N        North York   
23          M2P        North York   
24          M2R        North York   
25          M3A        North York   
26          M3B        North York   
27          M3C        North York   
28          M3H        North York   
29          M3J        North York   
30          M3K        North York   
31          M3L        North York   
32          M3M        North York   
33          M3N        North York   
34          M4A        North York   
35          M4B         East York   
36          M4C         East York   
37          M4E      East Toronto   
38          M4G         East York   
39          M4H         East York   
40          M4J         East York   
41          M4K      East Toronto   
42          M4L      East Toronto   
43          M4M      East Toronto   
44          M4N   Central Toronto   
45          M4P   Central Toronto   
46          M4R   Central Toronto   
47          M4S   Central Toronto   
48          M4T   Central Toronto   
49          M4V   Central Toronto   
50          M4W  Downtown Toronto   
51          M4X  Downtown Toronto   
52          M4Y  Downtown Toronto   
53          M5A  Downtown Toronto   
54          M5B  Downtown Toronto   
55          M5C  Downtown Toronto   
56          M5E  Downtown Toronto   
57          M5G  Downtown Toronto   
58          M5H  Downtown Toronto   
59          M5J  Downtown Toronto   
60          M5K  Downtown Toronto   
61          M5L  Downtown Toronto   
62          M5M        North York   
63          M5N   Central Toronto   
64          M5P   Central Toronto   
65          M5R   Central Toronto   
66          M5S  Downtown Toronto   
67          M5T  Downtown Toronto   
68          M5V  Downtown Toronto   
69          M5W  Downtown Toronto   
70          M5X  Downtown Toronto   
71          M6A        North York   
72          M6B        North York   
73          M6C              York   
74          M6E              York   
75          M6G  Downtown Toronto   
76          M6H      West Toronto   
77          M6J      West Toronto   
78          M6K      West Toronto   
79          M6L        North York   
80          M6M              York   
81          M6N              York   
82          M6P      West Toronto   
83          M6R      West Toronto   
84          M6S      West Toronto   
85          M7A  Downtown Toronto   
86          M7R       Mississauga   
87          M7Y      East Toronto   
88          M8V         Etobicoke   
89          M8W         Etobicoke   
90          M8X         Etobicoke   
91          M8Y         Etobicoke   
92          M8Z         Etobicoke   
94          M9B         Etobicoke   
95          M9C         Etobicoke   
96          M9L        North York   
97          M9M        North York   
98          M9N              York   
99          M9P         Etobicoke   
100         M9R         Etobicoke   
101         M9V         Etobicoke   
102         M9W         Etobicoke   

                                          Neighborhood   Latitude  Longitude  \
0                                      Malvern / Rouge  43.806686 -79.194353   
1             Rouge Hill / Port Union / Highland Creek  43.784535 -79

Lets Visualize the resulting cluster.

In [72]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Now examine the each cluster.

Cluster 1:

In [73]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
5        Scarborough               0            Playground   
14       Scarborough               0                  Park   
23        North York               0                  Park   
25        North York               0                  Park   
40         East York               0                  Park   
44   Central Toronto               0                  Park   
48   Central Toronto               0                  Park   
50  Downtown Toronto               0                  Park   
74              York               0                  Park   
90         Etobicoke               0                  Park   
98              York               0                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
5          Women's Store            Donut Shop    Dim Sum Restaurant   
14            Playground            Donut Shop    Dim Sum Restaurant   
23                  Bank                   Bar     Convenience Store   
25     Food & Drink Shop         Women's Store            Donut Shop   
40           Coffee Shop     Convenience Store         Women's Store   
44           Swim School              Bus Line      Doner Restaurant   
48            Playground           Summer Camp                 Trail   
50            Playground                 Trail     Electronics Store   
74                  Pool         Women's Store           Golf Course   
90                  Pool                 River      Department Store   
98     Convenience Store         Women's Store            Donut Shop   

          5th Most Common Venue 6th Most Common Venue  \
5                         Diner        Discount Store   
14                        Diner        Discount Store   
23                   Donut Shop        Discount Store   
25                        Diner        Discount Store   
40                   Donut Shop                 Diner   
44                        Diner        Discount Store   
48  Eastern European Restaurant   Dumpling Restaurant   
50  Eastern European Restaurant   Dumpling Restaurant   
74         Ethiopian Restaurant     Electronics Store   
90                 Dessert Shop    Dim Sum Restaurant   
98                        Diner        Discount Store   

          7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
5           Distribution Center               Dog Run      Doner Restaurant   
14          Distribution Center               Dog Run      Doner Restaurant   
23          Distribution Center               Dog Run      Doner Restaurant   
25          Distribution Center               Dog Run      Doner Restaurant   
40               Discount Store   Distribution Center               Dog Run   
44          Distribution Center               Dog Run            Donut Shop   
48                    Drugstore     Electronics Store            Donut Shop   
50          Empanada Restaurant             Drugstore            Donut Shop   
74  Eastern European Restaurant   Dumpling Restaurant             Drugstore   
90                        Diner        Discount Store   Distribution Center   
98          Distribution Center               Dog Run      Doner Restaurant   

   10th Most Common Venue  
5               Drugstore  
14              Drugstore  
23              Drugstore  
25              Drugstore  
40       Doner Restaurant  
44           Dessert Shop  
48          Deli / Bodega  
50          Deli / Bodega  
74             Donut Shop  
90                Dog Run  
98    Dumpling Restaurant

Cluster 2 :

In [74]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels      1st Most Common Venue  \
0         Scarborough               1       Fast Food Restaurant   
1         Scarborough               1                        Bar   
2         Scarborough               1        Rental Car Location   
3         Scarborough               1                Coffee Shop   
4         Scarborough               1       Caribbean Restaurant   
6         Scarborough               1           Department Store   
7         Scarborough               1                   Bus Line   
8         Scarborough               1        American Restaurant   
9         Scarborough               1                       Café   
10        Scarborough               1          Indian Restaurant   
11        Scarborough               1  Middle Eastern Restaurant   
12        Scarborough               1  Latin American Restaurant   
13        Scarborough               1                   Pharmacy   
15        Scarborough               1         Chinese Restaurant   
17         North York               1                       Pool   
18         North York               1             Clothing Store   
19         North York               1                       Bank   
20         North York               1                  Cafeteria   
22         North York               1                Pizza Place   
24         North York               1                Pizza Place   
26         North York               1                        Gym   
27         North York               1                        Gym   
28         North York               1                Coffee Shop   
29         North York               1         Falafel Restaurant   
30         North York               1              Grocery Store   
31         North York               1              Grocery Store   
32         North York               1              Grocery Store   
33         North York               1              Grocery Store   
34         North York               1      Portuguese Restaurant   
35          East York               1                Pizza Place   
36          East York               1                       Park   
37       East Toronto               1                      Trail   
38          East York               1        Sporting Goods Shop   
39          East York               1          Indian Restaurant   
41       East Toronto               1           Greek Restaurant   
42       East Toronto               1                       Park   
43       East Toronto               1                       Café   
45    Central Toronto               1                        Gym   
46    Central Toronto               1             Clothing Store   
47    Central Toronto               1             Sandwich Place   
49    Central Toronto               1                Coffee Shop   
51   Downtown Toronto               1                Coffee Shop   
52   Downtown Toronto               1                Coffee Shop   
53   Downtown Toronto               1                Coffee Shop   
54   Downtown Toronto               1             Clothing Store   
55   Downtown Toronto               1                Coffee Shop   
56   Downtown Toronto               1                Coffee Shop   
57   Downtown Toronto               1                Coffee Shop   
58   Downtown Toronto               1                Coffee Shop   
59   Downtown Toronto               1                Coffee Shop   
60   Downtown Toronto               1                Coffee Shop   
61   Downtown Toronto               1                Coffee Shop   
62         North York               1             Sandwich Place   
63    Central Toronto               1             Ice Cream Shop   
64    Central Toronto               1                      Trail   
65    Central Toronto               1                       Café   
66   Downtown Toronto               1                       Café   
67   Downtown Toronto               1                       Café   
68

Cluster 3 :

In [75]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
91   Etobicoke               2        Baseball Field         Women's Store   
97  North York               2          Food Service        Baseball Field   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
91            Donut Shop                 Diner        Discount Store   
97         Women's Store            Donut Shop        Discount Store   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
91   Distribution Center               Dog Run      Doner Restaurant   
97   Distribution Center               Dog Run      Doner Restaurant   

   9th Most Common Venue 10th Most Common Venue  
91             Drugstore         Farmers Market  
97             Drugstore     Dim Sum Restaurant

Cluster 4 :

In [76]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
94  Etobicoke               3           Golf Course         Women's Store   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
94             Drugstore                 Diner        Discount Store   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
94   Distribution Center               Dog Run      Doner Restaurant   

   9th Most Common Venue 10th Most Common Venue  
94            Donut Shop    Dumpling Restaurant

Cluster 5 :

In [77]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
21  North York               4             Piano Bar         Women's Store   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
21            Donut Shop    Dim Sum Restaurant                 Diner   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
21        Discount Store   Distribution Center               Dog Run   

   9th Most Common Venue 10th Most Common Venue  
21      Doner Restaurant              Drugstore

### Thank You --AKSHAY DILIP GODSE.